In [3]:
import os, requests

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver

ModuleNotFoundError: No module named 'selenium'

In [51]:
nl_files_list = os.listdir("../met-naam/castles-netherlands/")

In [59]:
nl_urls_list = ['https://www.'+item[:-6].lstrip('www')+'.'+item.rstrip('.txt')[-2:] for item in nl_files_list]

In [96]:
nl_urls_list[2]

### BeautifulSoup

In [97]:
response = requests.get(nl_urls_list[2])

In [93]:
soup = BeautifulSoup(response.text)

In [88]:
# get a list of all h1 tags based on css selectors
#titles = soup.select("h1,h2,h3,h4,h5,h6")
titles = soup.select("#main div bg section-bg fill bg-fill bg-loaded")
for t in titles:
  print(t.text)

In [103]:
# Find all text elements and concatenate them
allowed_tags = ['div','p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'span', 'br']
all_text = ' '.join([element.get_text() for element in soup.find_all(allowed_tags)])
with open(nl_files_list[2], "w") as file:
    file.write(all_text)
#print(all_text)

In [102]:
print(nl_files_list[2], nl_urls_list[2])

wwwmuiderslotnl.txt https://www.muiderslot.nl


In [104]:
url= "https://web.archive.org/web/20180511012246/https://www.muiderslot.nl/"

In [106]:
response1 = requests.get(url)

In [107]:
soup1 = BeautifulSoup(response.text)

In [108]:
allowed_tags = ['div','p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'span', 'br']
all_text = ' '.join([element.get_text() for element in soup.find_all(allowed_tags)])
with open(url[29:33], "w") as file:
    file.write(all_text)

### Selenium 

In [1]:
# open chrome
driver = webdriver.Chrome()

# visit alibaba
driver.get("https://www.alibaba.com/products/labor_camp.html?IndexArea=product_en&page=1")

# select h4 elements
items = driver.find_elements_by_css_selector("h4")

for i in items:
    print(i.text)

driver.close()

NameError: name 'webdriver' is not defined